## Load Data

In [ ]:
#Import initial packages and data.

import pandas as pd
import seaborn as sns
sns.set_context('notebook')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings as wrns
wrns.filterwarnings('ignore')
data = pd.read_csv('train.csv')
data.head()

## Missing Data Handling 

In [ ]:
#Seperate data by variable types.

Continuous = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
              '1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF',
              'EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','SalePrice']

Discrete = ['GarageYrBlt', 'YearRemod/Add', 'YearBuilt', 'BsmtFullBath','FullBath','HalfBath','BedroomAbvGr',
            'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','MoSold','YrSold']

Ordinal=['Utilities','LotShape','LandSlope','OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual',
         'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC','Electrical','KitchenQual',
         'Functional','FireplaceQu','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence']

Nominal = ['MSSubClass','MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood','Condition1',
           'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
           'MasVnrType','Foundation','Heating','CentralAir','GarageType','MiscFeature','SaleType']


In [ ]:
#Define function to find missing values.

def num_missing(x):
  return sum(x.isnull())

In [ ]:
#Find number of missing values for continuous variables.

print ('Missing values for continuous variables:')
missing_con = pd.DataFrame(data[Continuous].apply(num_missing, axis=0)[data[Continuous].apply(num_missing, axis=0)>0])
missing_con

In [ ]:
#Find number of missing values for discrete variables.

print ('Missing values for discrete variables:')
missing_dis = pd.DataFrame(data[Discrete].apply(num_missing, axis=0)[data[Discrete].apply(num_missing, axis=0)>0])
missing_dis

In [ ]:
#Find number of missing values for ordinal variables.

print ('Missing values for ordinal variables:')
missing_ord = pd.DataFrame(data[Ordinal].apply(num_missing, axis=0)[data[Ordinal].apply(num_missing, axis=0)>0])
missing_ord

In [ ]:
#Find number of missing values for nominal variables.

print ('Missing values for nominal variables:')
missing_nom = pd.DataFrame(data[Nominal].apply(num_missing, axis=0)[data[Nominal].apply(num_missing, axis=0)>0])
missing_nom

In [ ]:
#Filter out rows where MasVnrArea data is missing arbitrarily. This calls into question the entire row.

data = data[data['MasVnrArea'] >= 0]

#Fill object type blanks with NA or 0 as appropriate. Doing one by one instead of loop to easily see all altered variables.
#GarageYrBlt left as is and will be dealt with at later stage.

data['LotFrontage'] = data['LotFrontage'].fillna(0)
data['Alley'] = data['Alley'].fillna('NA')
data['BsmtQual'] = data['BsmtQual'].fillna('NA')
data['BsmtCond'] = data['BsmtCond'].fillna('NA')
data['BsmtExposure'] = data['BsmtExposure'].fillna('NA')
data['BsmtFinType1'] = data['BsmtFinType1'].fillna('NA')
data['BsmtFinType2'] = data['BsmtFinType2'].fillna('NA')
data['FireplaceQu'] = data['FireplaceQu'].fillna('NA')
data['GarageType'] = data['GarageType'].fillna('NA')
data['GarageFinish'] = data['GarageFinish'].fillna('NA')
data['GarageQual'] = data['GarageQual'].fillna('NA')
data['GarageCond'] = data['GarageCond'].fillna('NA')
data['PoolQC'] = data['PoolQC'].fillna('NA')
data['Fence'] = data['Fence'].fillna('NA')
data['MiscFeature'] = data['MiscFeature'].fillna('NA')

## Exploratory Data Analysis and Data Processing

In [ ]:
#Import statistical packages for later use.

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
#Convert MSSubClass to string since it is nominal but read in to python as numerical.

data['MSSubClass'] = data['MSSubClass'].astype(str)

#Create variables for age of house and age of garage to use instead of YearBlt, YearRemod/Add and GarageYrBlt.

data['AgeHouse'] = (data['YrSold'] - data['YearRemod/Add']).astype(float)
data['AgeGarage'] = (data['YrSold'] - data['GarageYrBlt']).astype(float)

#Replace AgeGarage blanks with 0 and append variables to continuous list.

data['AgeGarage'] = data['AgeGarage'].fillna(0)
Continuous.append('AgeHouse')
Continuous.append('AgeGarage')
del Discrete[0]

#### Continuous variables

In [ ]:
#Create table to describe continous variables.

table = data[Continuous].describe()
table.loc['skewness']= data.skew()
table.loc['kurtosis']= data.kurt()
table.round(2)

In [ ]:
#Create histograms and box plots for continuous variables and variables treated as continuous. Then save graphs as pictures.

for i in Continuous:
    fig,ax=plt.subplots(1,2,figsize=(12,5))
    sns.distplot(data[i],ax=ax[0],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
    ax[0].set(ylabel='Frequency', title='Histogram for {}'.format(i), xlabel=i)
    sns.boxplot(data[i],orient='v',ax=ax[1])
    ax[1].set(title='Box plot for {}'.format(i), ylabel=i)
    #fig.savefig('{}.png'.format(i))

In [ ]:
#Create LogSalePrice transformations for suspected right skew in SalePrice.

data['LogSalePrice'] = np.log(data['SalePrice'])

In [ ]:
#Print skew values before and after log transformation.

skew_std_err = 6/np.sqrt(len(data))
print('standard error of skewness = {}'.format(skew_std_err))
skew_before = data['SalePrice'].skew()
skew_after = data['LogSalePrice'].skew()
print('skew before log transformation = {}'.format(skew_before))
print('skew after log transformation = {}'.format(skew_after))

In [ ]:
#Remove anything that is more than 3 standard deviations from the mean to correct for outliers.
#Done on Sale Price rather than LogSalePrice.

data_removed_outliers = data[data['SalePrice']<500000]

In [ ]:
#Print skew after outliers removed and LogSalePrice used. Skewness almost 0.

skew_after = data_removed_outliers['LogSalePrice'].skew()
print('skew after log transformation and removed outliers = {}'.format(skew_after))

In [ ]:
#Histograms of SalePrice before removing outliers (initial) and LogSalePrice after removing outliers (final).
#Save results as picture.

fig,ax=plt.subplots(1,2,figsize=(12,5))
sns.distplot(data['SalePrice'],ax=ax[0],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[0].set(title='Histogram for SalePrice before removed outliers', xlabel='SalePrice', ylabel='Frequency')
sns.distplot(data_removed_outliers['LogSalePrice'],ax=ax[1],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[1].set(title='Histogram for LogSalePrice after removed outliers',  xlabel='logSalePrice', ylabel='Frequency')
#fig.savefig('histogram for saleprice before and after removed outliers.png')

In [ ]:
#Histogram of SalePrice and LogSalePrice before removed outliers.

fig,ax=plt.subplots(1,2,figsize=(12,5))
sns.distplot(data['SalePrice'],ax=ax[0],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[0].set(title='Histogram for SalePrice before removed outliers', xlabel=i)
sns.distplot(data['LogSalePrice'],ax=ax[1],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[1].set(title='Histogram for LogSalePrice before removed outliers', ylabel=i)
#fig.savefig('histogram for saleprice before and after removed outliers.png')

In [ ]:
#Histogram of SalePrice and LogSalePrice after removed outliers.

fig,ax=plt.subplots(1,2,figsize=(12,5))
sns.distplot(data['LogSalePrice'],ax=ax[0],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[0].set(title='Histogram for LogSalePrice before removing outliers', xlabel='LogSalePrice')
sns.distplot(data_removed_outliers['LogSalePrice'],ax=ax[1],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
ax[1].set(title='Histogram for LogSalePrice after removing outliers', xlabel='logSalePrice')
# fig.savefig('Histogram for SalePrice and logSalePrice.png'

In [ ]:
#Set transformed data set, data_removed_outliers, as main data.

data = data_removed_outliers

In [ ]:
#Draw regression plots for all continuous variables and save the results as picture.
#Some variables which have a linear relationship with SalePrice are MasVnrArea, TotalBsmtSF, 1stFlrSF, GrLivArea, GarageArea.
#Some variables which have a non-linear relationship with SalePrice are BsmtFinSF1, OpenporchSF.
#There could still be some outliers.

for i in Continuous:
    fig,ax=plt.subplots(1,1,figsize=(12,5))
    sns.regplot(data[i], data['LogSalePrice'], scatter_kws = {'s': 25}, lowess=True, color=sns.color_palette('Blues')[-1])
    sns.despine()
    ax.set(ylabel='LogSalePrice', title='Scatter plot for {}'.format(i))
    plt.show()
    #fig.savefig('{}.png'.format(i))

In [ ]:
#Print number of non-zero values for continuous variables.
#LowQualFinSF, 3SsnPorch, PoolArea, MiscVal have only a little non-zero values so the sample size is insufficient.

print('the number of non-zero value for each continuous varible')
for i in Continuous:
    count = 0
    for m in data[i]:
        if m != 0:
            count += 1
    print('{}:{}'.format(i,count))

#### Discrete variables

In [ ]:
#Plot histograms and box plots for discrete variables and save output as picture.

for i in Discrete:
    fig,ax=plt.subplots(1,2,figsize=(12,5))
    ax[0].set(title='Histogram for {}'.format(i), xlabel=i)
    sns.distplot(data[i],ax=ax[0],hist_kws={'alpha':0.9},kde_kws={'color':'black','alpha':0.6})
    ax[1].set(title='Boxplot for {}'.format(i), xlabel=i)
    sns.boxplot(data[i],orient='v',ax=ax[1])
    #fig.savefig('{}.png'.format(i))

In [ ]:
#Create table to describe data for discrete variables.

table = data[Discrete].describe()
table.loc['skewness']= data.skew()
table.loc['kurtosis']= data.kurt()
table.round(2)

In [ ]:
#Create regression plots of discrete variables and save output.
#There is an approximately linear trend for almost all discrete variables, except for YrSold, MoSold (as expected) and KitchenAbvGr.
#FullBath, GarageCars, TotRmsAbrGrd, BedroomAbvGr have non-linear trends in the box plots. 
#However, the sample size for some classes is insufficient. 
#Those variables are likely to have linear relationship with saleprice, if sample size is large enough.

for i in Discrete:
    fig,ax=plt.subplots(1,1,figsize=(12,5))
    sns.regplot(data[i], data['LogSalePrice'], scatter_kws = {'s': 25}, lowess=True, color=sns.color_palette('Blues')[-1])
    sns.despine()
    ax.set(ylabel='LogSalePrice', title = 'Scatter plot for {}'.format(i))
    plt.show()
    #fig.savefig('{}.png'.format(i))

In [ ]:
#Category count for those discrete variables that don't seem to have linear trends.

def count(cata_var):
    count = {}
    for i in cata_var.unique():
        count[i] = 0 
        for j in cata_var:
            if j == i:
                count[i] += 1
    return count

non_linear_discrete = ['FullBath', 'GarageCars', 'TotRmsAbvGrd', 'BedroomAbvGr']
for i in non_linear_discrete:
    print('{}:{}'.format(i,count(data[i])))

#### Ordinal Variables

In [ ]:
#Draw box plots for ordinal variables and save output as picture.
#There is linear trend for Quality variables (OverallQual, KitchenQual, FireplaceQC, etc.)
#Treat them as numerical rather than nominal or ordinal.

for i in Ordinal:
    fig, ax= plt.subplots()
    sns.boxplot(data[i], data['LogSalePrice'], ax=ax, palette='Blues')
    ax.set(ylabel='LogSalePrice', title='Boxplot for {}'.format(i))
    plt.tight_layout()
    plt.show()
    #fig.savefig('{}.png'.format(i))

In [ ]:
#Box plots of all "quality" variables and graphed in ascending order and saved as pictures.

Quality_terms = ['ExterQual', 'BsmtQual', 'BsmtExposure', 'KitchenQual', 'FireplaceQu', 
                 'GarageQual', 'HeatingQC', 'GarageCond', 'BsmtCond', 'ExterCond']
Order1 = ['NA','Po','Fa','TA','Gd','Ex']
Order2 = ['NA','No','Mn','Av','Gd']
for i in Quality_terms:
    if i != 'BsmtExposure':
        fig, ax= plt.subplots()
        sns.boxplot(data[i], data['LogSalePrice'], ax=ax, order = Order1, palette='Blues')
        ax.set(ylabel='LogSalePrice', title='Boxplot for {}'.format(i))
        plt.tight_layout()
        plt.show()
        #fig.savefig('{} in order.png'.format(i))
    if i == 'BsmtExposure':
        fig, ax= plt.subplots()
        sns.boxplot(data[i], data['LogSalePrice'], ax=ax, order = Order2, palette='Blues')
        ax.set(ylabel='LogSalePrice', title='Boxplot for {}'.format(i))
        plt.tight_layout()
        plt.show()
        #fig.savefig('{} in order.png'.format(i))

In [ ]:
#Count data breakdown by catagory/class.
#For some classes in a variable, the sample size is insufficient.

for i in Ordinal:
    print('{}:{}'.format(i,count(data[i])))

### Nominal Variables

In [ ]:
#Box plots of remaining nominal variables.

for i in Nominal:
    fig, ax= plt.subplots()
    sns.boxplot(data[i], data['LogSalePrice'], data=data, ax=ax, palette='Blues')
    ax.set(ylabel='LogSalePrice', title='Boxplot for {}'.format(i))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    #fig.savefig('boxplots for nominal {}.png'.format(i))

In [ ]:
#Count data breakdown by category.

for i in Nominal:
    print('{}:{}'.format(i,count(data[i])))

In [ ]:
#Convert month to string because it is stored as numerical, but not treated as numerical.

data['MoSold'] = data['MoSold'].astype(str) 

#Delete year built, remodeled, and garage year built.

del data['YearBuilt']
del data['YearRemod/Add']
del data['GarageYrBlt']

In [ ]:
#Duplicate data to keep record of previous version in case need to use it.

data_num = data.copy()

In [ ]:
#Sort neighborhoods by mean.

neighbor_mean = data_num.groupby('Neighborhood')[['SalePrice']].apply(np.mean)
neighbor_mean.sort_values('SalePrice')

In [ ]:
#Import scip to compare Neighborhood Categories with Welch's t-tests assuming unequal variance.
#Group categories accordingly to decrease number of neighborhood categories.
#Significance level 10%

from scipy import stats

#Leave MeadowV

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='MeadowV','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='BrDale','SalePrice'],equal_var=False)
print('MeadowV vs BrDale',pvalue)

#Combine BrDale and IDOTRR

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='BrDale','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='IDOTRR','SalePrice'],equal_var=False)
print('BrDale vs IDOTRR',pvalue)

#Combine OldTown Edwards SWISU BrkSide

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='OldTown','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Edwards','SalePrice'],equal_var=False)
print('OldTown vs Edwards',pvalue) 
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='OldTown','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='SWISU','SalePrice'],equal_var=False)
print('OldTown vs SWISU',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='OldTown','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='BrkSide','SalePrice'],equal_var=False)
print('OldTown vs BrkSide',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Edwards','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='SWISU','SalePrice'],equal_var=False)
print('Edwards vs SWISU',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Edwards','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='BrkSide','SalePrice'],equal_var=False)
print('Edwards vs BrkSide',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='SWISU','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='BrkSide','SalePrice'],equal_var=False)
print('SWISU vs BrkSide',pvalue)

#Combine Sawyer NAmes Blueste NPkVill

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Sawyer','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NAmes','SalePrice'],equal_var=False)
print('Sawyer vs NAmes',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Sawyer','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Blueste','SalePrice'],equal_var=False)
print('Sawyer vs Blueste',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Sawyer','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NPkVill','SalePrice'],equal_var=False)
print('Sawyer vs NPkVill',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='NAmes','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Blueste','SalePrice'],equal_var=False)
print('NAmes vs Blueste',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='NAmes','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NPkVill','SalePrice'],equal_var=False)
print('NAmes vs NPkVill',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Blueste','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NPkVill','SalePrice'],equal_var=False)
print('Blueste vs NPkVill',pvalue)

#Combine Mitchel SawyerW Blmngtn

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Mitchel','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='SawyerW','SalePrice'],equal_var=False)
print('Mitchel vs SawyerW',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Mitchel','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Blmngtn','SalePrice'],equal_var=False)
print('Mitchel vs Blmngtn',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='SawyerW','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Blmngtn','SalePrice'],equal_var=False)
print('SawyerW vs Blmgtn',pvalue)

#Combine Gilbert NWAmes Crawfor CollgCr

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Gilbert','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NWAmes','SalePrice'],equal_var=False)
print('Gilbert vs NWAmes',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Gilbert','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Crawfor','SalePrice'],equal_var=False)
print('Gilbert vs Crawfor',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Gilbert','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='CollgCr','SalePrice'],equal_var=False)
print('Gilbert vs CollgCr',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='NWAmes','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Crawfor','SalePrice'],equal_var=False)
print('NWAmes vs Crawfor',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='NWAmes','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='CollgCr','SalePrice'],equal_var=False)
print('NWAmes vs CollgCr',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Crawfor','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='CollgCr','SalePrice'],equal_var=False)
print('Crawfor vs CollgCr',pvalue)

#Combine Greens ClearCr Somerst Timber

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Greens','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='ClearCr','SalePrice'],equal_var=False)
print('Greens vs ClearCr',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Greens','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Somerst','SalePrice'],equal_var=False)
print('Greens vs Somerst',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Greens','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Timber','SalePrice'],equal_var=False)
print('Greens vs Timber',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='ClearCr','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Somerst','SalePrice'],equal_var=False)
print('ClearCr vs Somerst',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='ClearCr','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Timber','SalePrice'],equal_var=False)
print('ClearCr vs Timber',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Somerst','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='Timber','SalePrice'],equal_var=False)
print('Somerst vs Timber',pvalue)

#Leave Veenker

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='Veenker','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='StoneBr','SalePrice'],equal_var=False)
print('Veenker vs StoneBr',pvalue)

#Combine StoneBr NoRidge NridgHt

t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='StoneBr','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NoRidge','SalePrice'],equal_var=False)
print('StoneBr vs NoRidge',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='StoneBr','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NridgHt','SalePrice'],equal_var=False)
print('StoneBr vs NridgHt',pvalue)
t, pvalue = stats.ttest_ind(data_num.loc[data_num['Neighborhood']=='NoRidge','SalePrice'],
                            data_num.loc[data_num['Neighborhood']=='NridgHt','SalePrice'],equal_var=False)
print('NoRidge vs NridgHt',pvalue)

#### Correlation Matrix

In [ ]:
#Show top 25 numerical variables by correlation to LogSalePrice.

pd.set_option('display.max_rows',300)
abs_correl = abs(data_num.corr().round(3)['LogSalePrice'])
abs_correl_sort = abs_correl.sort_values(ascending=False)
abs_correl_sort.head(27)
pd.DataFrame(abs_correl_sort.head(27))

In [ ]:
#Get list so easy to copy and paste into cell below. Use 27 to get top 25 for correlation heatmap.

abs_correl_sort.head(27).index

In [ ]:
#Put top 25 numerical variables in a list.

top_variables_25 = ['LogSalePrice', 'OverallQual', 'GrLivArea', 'GarageCars',
       'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'AgeHouse', 'FullBath',
       'Fireplaces', 'TotRmsAbvGrd', 'BsmtFinSF1', 'AgeGarage', 'MasVnrArea',
       'WoodDeckSF', 'HalfBath', 'OpenPorchSF', '2ndFlrSF', 'LotArea',
       'BsmtFullBath', 'BedroomAbvGr', 'BsmtUnfSF', 'EnclosedPorch',
       'KitchenAbvGr', 'LotFrontage', 'ScreenPorch']

In [ ]:
#Create heatmap of top 25 numerical variables. Note this is before dummy variable creation and simply to have an idea of
#numerical relationships.

data_num[top_variables_25].corr().round(4)
fig, ax = plt.subplots(figsize = (11,10))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[top_variables_25].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for top 25 variables correlated with LogSalePrice')
#fig.savefig('Correlation heatmap before feature engineering (top 25).png')

## Feature Engineering

In [ ]:
#Combine Neighborhood variables using results from Welch's t-tests in EDA.

neighbor_combined = {'MeadowV':'MeadowV',
                     'BrDale': 'BrD_IDO',
                     'IDOTRR': 'BrD_IDO',
                     'OldTown':'Old_Ed_SW_Brk',
                     'Edwards':'Old_Ed_SW_Brk',
                     'SWISU':'Old_Ed_SW_Brk',
                     'BrkSide':'Old_Ed_SW_Brk',
                     'Sawyer':'Sa_NA_Bl_NP',
                     'NAmes':'Sa_NA_Bl_NP',
                     'Blueste':'Sa_NA_Bl_NP',
                     'NPkVill':'Sa_NA_Bl_NP',
                     'Mitchel':'Mi_SaW,Bng',
                     'SawyerW':'Mi_SaW,Bng',
                     'Blmngtn':'Mi_SaW,Bng',
                     'Gilbert':'Gi_NWA_Cr_Co',
                     'NWAmes':'Gi_NWA_Cr_Co',
                     'Crawfor':'Gi_NWA_Cr_Co',
                     'CollgCr':'Gi_NWA_Cr_Co',
                     'Greens':'Gr_CC_So_Ti',
                     'ClearCr':'Gr_CC_So_Ti',
                     'Somerst':'Gr_CC_So_Ti',
                     'Timber':'Gr_CC_So_Ti',
                     'Veenker':'Veenker',
                     'StoneBr':'St_No_NHt',
                     'NoRidge':'St_No_NHt',
                     'NridgHt':'St_No_NHt'}
data_num['Neighborhood'] = data_num['Neighborhood'].map(neighbor_combined)

In [ ]:
#Box plots of neighborhood after combining neighborhoods.

fig, ax = plt.subplots(figsize=(10,10))
sns.boxplot(x = 'Neighborhood' , y = 'LogSalePrice', ax = ax, data = data_num, palette = 'Greens')
plt.xticks(rotation=20)
ax.set(title='Boxplot for Neighborhood after clustering')
#fig.savefig('Boxplot for Neighborhood after clustering.png')
plt.show()

In [ ]:
#Recode and convert quality type ordinals to numerical for more simplicity in analysis.

exterqual_vals = {'Fa':1,'TA':2,'Gd':3,'Ex':4}
data_num['ExterQual'] = data_num['ExterQual'].map(exterqual_vals).astype(int)

bsmt_vals = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_num['BsmtQual'] = data_num['BsmtQual'].map(bsmt_vals).astype(int)

bsmt_exp = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
data_num['BsmtExposure'] = data_num['BsmtExposure'].map(bsmt_exp).astype(int)

kitchen_vals = {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_num['KitchenQual'] = data_num['KitchenQual'].map(kitchen_vals).astype(int)

fireplace_qual = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_num['FireplaceQu'] = data_num['FireplaceQu'].map(fireplace_qual).astype(int)

garage_vals= {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4}
data_num['GarageQual'] = data_num['GarageQual'].map(garage_vals).astype(int)

heating_vals = {'Fa':1,'TA':2,'Gd':3,'Ex':4}
data_num['HeatingQC'] = data_num['HeatingQC'].map(kitchen_vals).astype(int)

garage_cond = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4}
data_num['GarageCond'] = data_num['GarageCond'].map(garage_cond).astype(int)

bsmt_cond = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_num['BsmtCond'] = data_num['BsmtCond'].map(bsmt_cond).astype(int)

exter_cond = {'Fa':1,'TA':2,'Gd':3,'Ex':4}
data_num['ExterCond'] = data_num['ExterCond'].map(exter_cond).astype(int)

In [ ]:
#Create dummy variables for remaining categoricals.

columns_cat_num = list(data_num.select_dtypes(['object']).columns)
for column in columns_cat_num:
    data_num = pd.get_dummies(data_num, columns = [column])

In [ ]:
#Create interactions. GrLiv_Rooms created but ultimately left out.

data_num['BsmtSF_Qual'] = data_num['TotalBsmtSF']*data_num['BsmtQual']
data_num['GarageArea_Qual'] = data_num['GarageArea']*data_num['GarageQual']
#data_num['GrLiv_Rooms'] = data_num['GrLivArea']*data_num['TotRmsAbvGrd']
data_num['GarageArea_Cars'] = data_num['GarageArea']*data_num['GarageCars']

## Further EDA - Correlation Matrix with Interaction Terms

In [ ]:
#Show top 70 correlated with LogSalePrice

pd.set_option('display.max_rows',300)
abs_correl = abs(data_num.corr().round(3)['LogSalePrice'])
abs_correl_sort = abs_correl.sort_values(ascending=False)
abs_correl_sort.head(72)

In [ ]:
#Print list so easy to copy and paste in cell below.

abs_correl_sort.head(72).index

In [ ]:
#List used for large 70 variable correlation heatmap.

predictors_heat_large = ['LogSalePrice', 'OverallQual', 'BsmtSF_Qual', 'GrLivArea',
       'GarageCars', 'GarageArea_Cars', 'GarageArea_Qual', 'GarageArea',
       'ExterQual', 'BsmtQual', 'TotalBsmtSF', 'KitchenQual', '1stFlrSF',
       'AgeHouse', 'FullBath', 'FireplaceQu', 'Foundation_PConc', 'Fireplaces',
       'TotRmsAbvGrd', 'Neighborhood_St_No_NHt', 'HeatingQC',
       'GarageType_Attchd', 'GarageFinish_Unf', 'BsmtFinType1_GLQ',
       'GarageType_Detchd', 'BsmtFinSF1', 'AgeGarage',
       'Neighborhood_Old_Ed_SW_Brk', 'MSSubClass_60', 'MasVnrArea',
       'CentralAir_Y', 'CentralAir_N', 'PavedDrive_Y', 'Exterior1st_VinylSd',
       'Exterior2nd_VinylSd', 'GarageQual', 'BsmtExposure', 'PavedDrive_N',
       'MSZoning_RM', 'WoodDeckSF', 'GarageFinish_Fin', 'LotShape_Reg',
       'GarageCond', 'MasVnrType_None', 'HalfBath', 'MSSubClass_30',
       'OpenPorchSF', 'Electrical_SBrkr', 'LotShape_IR1', 'Foundation_CBlock',
       '2ndFlrSF', 'GarageType_NA', 'GarageFinish_NA', 'BsmtCond',
       'MSZoning_RL', 'GarageFinish_RFn', 'LotArea', 'MasVnrType_BrkFace',
       'Foundation_BrkTil', 'HouseStyle_2Story', 'Neighborhood_Gr_CC_So_Ti',
       'BsmtFullBath', 'Electrical_FuseA', 'Neighborhood_Sa_NA_Bl_NP',
       'BsmtFinType1_NA', 'BsmtFinType2_NA', 'MasVnrType_Stone',
       'Neighborhood_Gi_NWA_Cr_Co', 'Neighborhood_BrD_IDO', 'BedroomAbvGr',
       'Fence_NA']

#List used for smaller 30 variable correlation heatmap.

predictors_heat = ['LogSalePrice', 'OverallQual', 'BsmtSF_Qual', 'GrLivArea',
                   'GarageCars', 'GarageArea_Cars', 'GarageArea_Qual', 'GarageArea',
                   'ExterQual', 'BsmtQual', 'TotalBsmtSF', 'KitchenQual', '1stFlrSF',
                   'AgeHouse', 'FullBath', 'FireplaceQu', 'Foundation_PConc', 'Fireplaces',
                   'TotRmsAbvGrd', 'Neighborhood_St_No_NHt', 'HeatingQC',
                   'GarageType_Attchd', 'GarageFinish_Unf', 'BsmtFinType1_GLQ',
                   'GarageType_Detchd', 'BsmtFinSF1', 'AgeGarage',
                   'Neighborhood_Old_Ed_SW_Brk', 'MSSubClass_60', 'MasVnrArea',
                   'CentralAir_Y']


#Predictors used for OLS model.
#Started with 30 and removed GarageArea_Qual, Fireplaces, TotRmsAbvGrd, GarageType_Detchd, 1stFlrSF.

predictors1 = ['OverallQual', 'BsmtSF_Qual', 'GrLivArea', 'GarageCars', 'GarageArea_Cars', 
                'GarageArea', 'ExterQual', 'BsmtQual', 'TotalBsmtSF', 'KitchenQual',
                'AgeHouse', 'FullBath', 'FireplaceQu', 'Foundation_PConc','Neighborhood_St_No_NHt', 
                'HeatingQC', 'GarageType_Attchd', 'GarageFinish_Unf', 'BsmtFinType1_GLQ',
                'BsmtFinSF1', 'AgeGarage', 'Neighborhood_Old_Ed_SW_Brk', 'MSSubClass_60', 
                'MasVnrArea', 'CentralAir_Y']

#Predictors used for Fwd + Ridge, Lasso, Enet, and Fwd + PCR models.
#Started with 70 and removed GarageArea_Qual, Fireplaces, TotRmsAbvGrd, GarageType_Detchd, 1stFlrSF, CentralAir_N, PavedDrive_N, 
#GarageCond, LotShape_IR1, Foundation_CBlock, 2ndFlrSF, GarageType_NA, GarageFinish_NA, BsmtCond, MSZoning_RL, BsmtFullBath
#MasVnrType_BrkFace, HouseStyle_2Story, Electrical_FuseA

predictors2 = ['OverallQual', 'BsmtSF_Qual', 'GrLivArea',
       'GarageCars', 'GarageArea_Cars', 'GarageArea',
       'ExterQual', 'BsmtQual', 'TotalBsmtSF', 'KitchenQual',
       'AgeHouse', 'FullBath', 'FireplaceQu', 'Foundation_PConc',
       'Neighborhood_St_No_NHt', 'HeatingQC',
       'GarageType_Attchd', 'GarageFinish_Unf', 'BsmtFinType1_GLQ',
       'BsmtFinSF1', 'AgeGarage',
       'Neighborhood_Old_Ed_SW_Brk', 'MSSubClass_60', 'MasVnrArea',
       'CentralAir_Y', 'PavedDrive_Y', 'Exterior1st_VinylSd',
       'Exterior2nd_VinylSd', 'GarageQual', 'BsmtExposure',
       'MSZoning_RM', 'WoodDeckSF', 'GarageFinish_Fin', 'LotShape_Reg',
       'MasVnrType_None', 'HalfBath', 'MSSubClass_30',
       'OpenPorchSF', 'Electrical_SBrkr',
       'GarageFinish_RFn', 'LotArea',
       'Foundation_BrkTil', 'Neighborhood_Gr_CC_So_Ti',
       'Neighborhood_Sa_NA_Bl_NP',
       'BsmtFinType1_NA', 'BsmtFinType2_NA', 'MasVnrType_Stone',
       'Neighborhood_Gi_NWA_Cr_Co', 'Neighborhood_BrD_IDO', 'BedroomAbvGr',
       'Fence_NA']

In [ ]:
#Display correlations of top 70.

pd.set_option('display.max_columns',71)
data_num[predictors_heat_large].corr().round(3)

In [ ]:
#Display large heat map and save figure to have a large visual idea of variable relationships.

data_num[predictors_heat_large].corr().round(4)
fig, ax = plt.subplots(figsize = (14,14))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[predictors_heat_large].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for top 70 variables correlated with LogSalePrice')
#fig.savefig('Correlation heatmap after feature engineering (top 70).png')

In [ ]:
#Display small heatmap and save figure.

data_num[predictors_heat].corr().round(4)
fig, ax = plt.subplots(figsize = (14,14))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[predictors_heat].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for top 30 variables correlated with LogSalePrice')
#fig.savefig('Correlation heatmap after feature engineering (top 30).png')

## OLS (Submission 20, MAE = 14621.89)

In [ ]:
#Create copy of the data to standardize.

data_s = data_num.copy()
data_pred = data_num[predictors2] #This is so we can call the columns by dtype in the next cell to divide by 2 SD
data_s.head()

In [ ]:
#Standardize data by subtracting predictors by respective mean and dividing by one standard deviation for dummy variables and
#2 standard deviations for numerical variables. This is so they are on roughly the same scale for using the regularization
#methods. Suggested by Marcel as something to try (not necessarily required) and also cited in various literature.

pred_float = data_pred.select_dtypes(['float64']).columns
pred_int = data_pred.select_dtypes(['int64']).columns
pred_dum = data_pred.select_dtypes(['uint8']).columns

mu_float = np.mean(data_s[pred_float])
sigma_float = np.std(data_s[pred_float])

mu_int = np.mean(data_s[pred_int])
sigma_int = np.std(data_s[pred_int])

mu_dum = np.mean(data_s[pred_dum])
sigma_dum = np.std(data_s[pred_dum])

data_s[pred_float] = (data_s[pred_float]-mu_float)/(2*sigma_float)
data_s[pred_int] = (data_s[pred_int]-mu_int)/(2*sigma_int)
data_s[pred_dum] = (data_s[pred_dum])/(sigma_dum)

data_s.head()

In [ ]:
#Fit OLS model on 25 predictors and calculate cross validation score on LogSalePrice.

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

def ols_reg_10_folds(predictors, response):
    ols = LinearRegression()
    scores = cross_val_score(ols, data_s[predictors], data_s[response], cv=10, scoring = 'neg_mean_absolute_error')
    cv_mae = np.mean(-1*scores)
    return cv_mae

ols = LinearRegression()
ols.fit(data_s[predictors1],data_s['LogSalePrice'])

ols_reg_10_folds(predictors1,'LogSalePrice')

In [ ]:
#Display OLS coefficients.

pd.set_option('display.max_columns',30)
pd.DataFrame(ols.coef_.round(5), index = predictors1).T

In [ ]:
#Residual plots of OLS.

ols.fit(data_s[predictors1], data_s['LogSalePrice'])
y_fit = ols.predict(data_s[predictors1])
y_actual = data_s['LogSalePrice']
residuals = y_fit-y_actual
abs_residuals = abs(residuals)

fig, ax= plt.subplots(1,2, figsize=(13,5))
sns.regplot(y_fit, residuals, fit_reg=False, ax=ax[0], scatter_kws={'alpha':0.5})
ax[0].set_xlabel('Fitted values')
ax[0].set_ylabel('Residuals')
ax[0].set(title='Residuals vs Fitted Values')
sns.regplot(y_fit, abs_residuals, fit_reg=False, ax=ax[1], scatter_kws={'alpha':0.5, 'color': sns.color_palette()[0]})
ax[1].set_xlabel('Fitted values')
ax[1].set_ylabel('Absolute residuals')
ax[1].set(title='Absolure residuals vs Fitted Values')
sns.despine()
plt.tight_layout()
plt.show()
fig.savefig('residuals_OLS.png')

In [ ]:
#Heatmap to check for perfect multicollinearity. ALready done in previous steps when creating heatmaps, but checking again. 

data_s[predictors1].corr().round(4)
fig, ax = plt.subplots(figsize = (10,10))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_s[predictors1].corr(), ax=ax, cmap=cmap,center=0)
ax.set(title='Correlation heatmap for predictors used in OLS')
#fig.savefig('correlation_OLS.png')

## Lasso, Ridge, and Elastic Net

#### LASSO (Submission 21, MAE = 13277.61)

In [ ]:
#Linear Regression fit to compare to Lasso, Ridge, and ENet coefficents.

from sklearn.linear_model import LinearRegression
ols2 = LinearRegression()
ols2.fit(data_s[predictors2],data_s['LogSalePrice'])

#Lasso CV to select shrinkage ad fit Lasso on 51 variables.

from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5)
lasso.fit(data_s[predictors2], np.ravel(data_s['LogSalePrice']))

In [ ]:
#Ratio of Lasso coefficients to OLS coefficients

round(np.linalg.norm(lasso.coef_, ord=1)/np.linalg.norm(np.ravel(ols2.coef_), ord=1),10)

In [ ]:
#Display Lasso coefficients.

pd.set_option('display.max_columns',51)
pd.DataFrame(lasso.coef_.round(5), index = predictors2).T

In [ ]:
#Calculate Lasso MAE CV Score on LogSalePrice

scores_lasso = cross_val_score(lasso, data_s[predictors2], np.ravel(data_s['LogSalePrice']), 
                               cv=10, scoring = 'neg_mean_absolute_error')
cv_mae_lasso = np.mean(-1*scores_lasso)
cv_mae_lasso

In [ ]:
#Residual plots of Lasso

lasso.fit(data_s[predictors2], data_s['LogSalePrice'])
y_fit = lasso.predict(data_s[predictors2])
y_actual = data_s['LogSalePrice']
residuals = y_fit - y_actual
abs_residuals = abs(residuals)

fig, ax= plt.subplots(1,2, figsize=(13,5))
sns.regplot(y_fit, residuals, fit_reg=False, ax=ax[0], scatter_kws={'alpha':0.5})
ax[0].set_xlabel('Fitted values')
ax[0].set_ylabel('Residuals')
ax[0].set(title='Residuals vs Fitted Values')
sns.regplot(y_fit, abs_residuals, fit_reg=False, ax=ax[1], scatter_kws={'alpha':0.5, 'color': sns.color_palette()[0]})
ax[1].set_xlabel('Fitted values')
ax[1].set_ylabel('Absolute residuals')
ax[1].set(title='Absolute residuals vs Fitted Values')
sns.despine()
plt.tight_layout()
plt.show()
fig.savefig('residuals_lasso.png')

#### Fwd + Ridge (Submission 18, MAE = 13225.03)

In [ ]:
#Define function and class for forward selection algorithm taken from Marcel's QBUS2820 file and edited for MAE.

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

def forwardselection(X, y):
    """Forward variable selection based on the Scikit learn API
    
    
    Output:
    ----------------------------------------------------------------------------------
    Scikit learn OLS regression object for the best model
    """

    # Functions
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import cross_val_score

    # Initialisation
    base = []
    p = X.shape[1]
    candidates = list(np.arange(p))

    # Forward recursion
    i=1
    bestcvscore=-np.inf    
    while i<=p:
        bestscore = 0
        for variable in candidates:
            ols = LinearRegression()
            ols.fit(X.iloc[:, base + [variable]], y)
            score = ols.score(X.iloc[:, base + [variable]], y)
            if score > bestscore:
                bestscore = score 
                best = ols
                newvariable=variable
        base.append(newvariable)
        candidates.remove(newvariable)
        
        cvscore = cross_val_score(best, X.iloc[:, base], y, scoring='neg_mean_absolute_error').mean() 
        
        if cvscore > bestcvscore:
            bestcvscore=cvscore
            bestcv = best
            subset = base[:]
        i+=1
    
    #Finalise
    return bestcv, subset

class forward:
    def __init__(self):
        pass

    def fit(self, X, y):
        self.ols, self.subset = forwardselection(X, y)

    def predict(self, X):
        return self.ols.predict(X.iloc[:, self.subset])

    def cv_score(self, X, y, cv=10):
        from sklearn.model_selection import cross_val_score
        scores = cross_val_score(self.ols, X.iloc[:, self.subset], np.ravel(y), cv=cv, scoring='neg_mean_absolute_error')
        return -1*np.mean(scores)

In [ ]:
#Fit forward selection on 51 predicotrs and print index of chosen predictors.

fwd = forward()
fwd.fit(data_s[predictors2], data_s['LogSalePrice'])
forwardselection(data_s[predictors2],data_s['LogSalePrice'])

In [ ]:
#Copy and paste index from above output and run through loop to extract predictor names.
#Name chosen list of predictors to be predictors_fw

fwd_predictors_index = [0,2,1,19,10,3,30,24,40,15,12,29,25,9,11,42,47,14,36,20,5]
 
predictors_fw = []
for i in fwd_predictors_index:
    predictors_fw.append(predictors2[i])
print(predictors_fw)

In [ ]:
#Ridge CV to select shrinkage.

from sklearn.linear_model import RidgeCV
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(data_s[predictors_fw], np.ravel(data_s['LogSalePrice']))

In [ ]:
#Ratio of Ridge to OLS coefficients of forward selected predictors.

from sklearn.linear_model import LinearRegression
ols3 = LinearRegression()
ols3.fit(data_s[predictors_fw],data_s['LogSalePrice'])
print(round(np.linalg.norm(ridge.coef_)/np.linalg.norm(np.ravel(ols3.coef_)), 3))

In [ ]:
#Fit Ridge on forward selected predictors.

from sklearn.linear_model import Ridge
ridge_fw = Ridge(alpha=ridge.alpha_)
ridge_fw.fit(data_s[predictors_fw], np.ravel(data_s['LogSalePrice']))

In [ ]:
#Display ridge coefficients

pd.set_option('display.max_columns',50)
pd.DataFrame(ridge_fw.coef_.round(5), index = predictors_fw).T

In [ ]:
#Calculate Ridge MAE CV on LogSalePrice

scores_ridge = cross_val_score(ridge_fw, data_s[predictors_fw], np.ravel(data_s['LogSalePrice']), 
                               cv=10, scoring = 'neg_mean_absolute_error')
cv_mae_ridge = np.mean(-1*scores_ridge)
cv_mae_ridge

In [ ]:
#Residual plot of Ridge with forward selection

ridge_fw.fit(data_s[predictors2], data_s['LogSalePrice'])
y_fit = ridge_fw.predict(data_s[predictors2])
y_actual = data_s['LogSalePrice']
residuals = y_fit-y_actual
abs_residuals = abs(residuals)

fig, ax= plt.subplots(1,2, figsize=(13,5))
sns.regplot(y_fit, residuals, fit_reg=False, ax=ax[0], scatter_kws={'alpha':0.5})
ax[0].set_xlabel('Fitted values')
ax[0].set_ylabel('Residuals')
ax[0].set(title='Residuals vs Fitted Values')
sns.regplot(y_fit, abs_residuals, fit_reg=False, ax=ax[1], scatter_kws={'alpha':0.5, 'color': sns.color_palette()[0]})
ax[1].set_xlabel('Fitted values')
ax[1].set_ylabel('Absolute residuals')
ax[1].set(title='Absolute residuals vs Fitted Values')
sns.despine()
plt.tight_layout()
plt.show()
#fig.savefig('residuals_ridge_with_fwd.png')

In [ ]:
#Correlation heatmap on forward selected predictors to see relationships and check for perfect multicollinearity.

data_num[predictors_fw].corr().round(4)
fig, ax = plt.subplots(figsize = (14,14))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[predictors_fw].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for variables used in ridge regression (selected by forward selection)')
#fig.savefig('Correlation heatmap ridge.png')

#### ElasticNet (Submission 19, MAE = 13280.37)

In [ ]:
#Select ENet shrinkage

from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(data_s[predictors2],np.ravel(data_s['LogSalePrice']))

In [ ]:
#Fit ENet model on 51 predictors

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(data_s[predictors2],np.ravel(data_s['LogSalePrice']))

In [ ]:
#Ratio of ENet coefficients to OLS coefficients.

from sklearn.linear_model import LinearRegression
ols4 = LinearRegression()
ols4.fit(data_s[predictors2],data_s['LogSalePrice'])
print(round(np.linalg.norm(enet.coef_)/np.linalg.norm(np.ravel(ols4.coef_)), 3))

In [ ]:
#Display ENet coefficients.

pd.set_option('display.max_columns',51)
pd.DataFrame(enet.coef_.round(5), index = predictors2).T

In [ ]:
#List of ENet predictors.

enet_pred = ['OverallQual', 'BsmtSF_Qual', 'GrLivArea', 'GarageCars', 'GarageArea', 
           'BsmtQual', 'TotalBsmtSF', 'KitchenQual', 'AgeHouse', 'FireplaceQu', 
          'Neighborhood_St_No_NHt','HeatingQC','GarageType_Attchd', 'BsmtFinSF1', 
          'Neighborhood_Old_Ed_SW_Brk', 'CentralAir_Y', 'PavedDrive_Y', 'Exterior2nd_VinylSd',
          'GarageQual', 'BsmtExposure', 'MSZoning_RM', 'GarageFinish_Fin', 'LotShape_Reg',
          'MSSubClass_30','Electrical_SBrkr','LotArea', 'Neighborhood_Gr_CC_So_Ti', 'Neighborhood_Sa_NA_Bl_NP',
          'Neighborhood_Gi_NWA_Cr_Co', 'Neighborhood_BrD_IDO']

In [ ]:
#To calculate ENet MAE CV on LogSalePrice

scores_enet = cross_val_score(enet, data_s[predictors2], np.ravel(data_s['LogSalePrice']), 
                               cv=10, scoring = 'neg_mean_absolute_error')
cv_mae_enet = np.mean(-1*scores_enet)
cv_mae_enet

In [ ]:
#Residual plots of Elastic net

enet.fit(data_s[predictors2], data_s['LogSalePrice'])
y_fit = enet.predict(data_s[predictors2])
y_actual = data_s['LogSalePrice']
residuals = y_fit-y_actual
abs_residuals = abs(residuals)

fig, ax= plt.subplots(1,2, figsize=(13,5))
sns.regplot(y_fit, residuals, fit_reg=False, ax=ax[0], scatter_kws={'alpha':0.5})
ax[0].set_xlabel('Fitted values')
ax[0].set_ylabel('Residuals')
ax[0].set(title='Residuals vs Fitted Values')
sns.regplot(y_fit, abs_residuals, fit_reg=False, ax=ax[1], scatter_kws={'alpha':0.5, 'color': sns.color_palette()[0]})
ax[1].set_xlabel('Fitted values')
ax[1].set_ylabel('Absolute residuals')
ax[1].set(title='Absolute residuals vs Fitted Values')
sns.despine()
plt.tight_layout()
plt.show()
#fig.savefig('residuals_enet.png')

In [ ]:
#Heatmap of ENet predictors.

data_num[enet_pred].corr().round(4)
fig, ax = plt.subplots(figsize = (14,14))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[enet_pred].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for variables used in elastic net')
#fig.savefig('Correlation heatmap enet.png')

## PCR with Forward Selection (Submission 22, MAE = 20918.68)

In [ ]:
#PCR algorithms taken from Marcel's QBUS2820 file and edited for MAE.

class PCR:
    def __init__(self, M=1):
        self.M=M

    def fit(self, X, y):
        from sklearn.decomposition import PCA
        from sklearn.linear_model import LinearRegression
        
        self.pca=PCA(n_components=self.M)
        Z= self.pca.fit_transform(X)
        self.pcr = LinearRegression().fit(Z, y)

    def predict(self, X):
        return self.pcr.predict(self.pca.transform(X))

    def cv_score(self, X, y, cv=10):
        from sklearn.model_selection import cross_val_score
        Z=self.pca.transform(X)
        scores = cross_val_score(self.pcr, Z, np.ravel(y), cv=cv, scoring='neg_mean_absolute_error').mean() 
        return -1*np.mean(scores)


def pcrCV(X, y):
    # Approximate cross-validation
    from sklearn.model_selection import cross_val_score
    
    p=X.shape[1]
    bestscore= -np.inf
    cv_scores = []
    for m in range(1,p+1):
        model = PCR(M=m)
        model.fit(X, y)
        Z=model.pca.transform(X)
        score = cross_val_score(model.pcr, Z, y, cv=10, scoring='neg_mean_absolute_error').mean() 
        cv_scores.append(score)
        if score > bestscore:
            bestscore=score
            best=model

    best.cv_scores = pd.Series(cv_scores, index = np.arange(1,p+1))
    return best

In [ ]:
#Fit PCR model.
pcr = pcrCV(data_s[predictors_fw],data_s['LogSalePrice'])

In [ ]:
#To calculate PCR MAE CV on LogSalePrice.

cv_score_pcr = pcr.cv_score(data_s[predictors_fw], np.ravel(data_s['LogSalePrice']), cv=10)
cv_score_pcr

In [ ]:
#Residual plot of PCR

pcr.fit(data_s[predictors2], data_s['LogSalePrice'])
y_fit = pcr.predict(data_s[predictors2])
y_actual = data_s['LogSalePrice']
residuals = y_fit-y_actual
abs_residuals = abs(residuals)

fig, ax= plt.subplots(1,2, figsize=(13,5))
sns.regplot(y_fit, residuals, fit_reg=False, ax=ax[0], scatter_kws={'alpha':0.5})
ax[0].set_xlabel('Fitted values')
ax[0].set_ylabel('Residuals')
ax[0].set(title='Residuals vs Fitted Values')
sns.regplot(y_fit, abs_residuals, fit_reg=False, ax=ax[1], scatter_kws={'alpha':0.5, 'color': sns.color_palette()[0]})
ax[1].set_xlabel('Fitted values')
ax[1].set_ylabel('Absolute residuals')
ax[1].set(title='Absolute residuals vs Fitted Values')
sns.despine()
plt.tight_layout()
plt.show()
#fig.savefig('residuals_pcr.png')

## KNN (Submission 23, MAE = 52483.60)

In [ ]:
#Define KNN algorithm and combine with forward selection.

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

def knn_test(X, y):
    
    neighbours=np.arange(1, 40)
    best_score = -np.inf
    
    for k in neighbours: 
        knn = KNeighborsRegressor(n_neighbors = k, metric='mahalanobis', metric_params={'V': X.cov()}) 
        scores = cross_val_score(knn, X, y, cv=10, scoring = 'neg_mean_absolute_error')
        cv_score = np.mean(scores)
        if cv_score >= best_score:
            best_score = cv_score
            best_knn = knn
    
    knn = best_knn
    knn.fit(X, y)
    #predictions = knn.predict(test[predictors])
    #test_rmse = np.sqrt(mean_squared_error(test[response], predictions))
    #cv_rmse= np.sqrt(-best_score)
    cv_mae = -best_score
    print('Chosen K: {}'.format(knn.n_neighbors))
    #return test_rmse, cv_rmse 
    return cv_mae, knn

def select_knn(X, y):
    """Forward variable selection based on the Scikit learn API
    
    
    Output:
    ----------------------------------------------------------------------------------
    Scikit learn knn regression object for the best model
    """

    # Functions
    from sklearn.model_selection import cross_val_score

    # Initialisation
    base = []
    p = X.shape[1]
    candidates = list(np.arange(p))

    # Forward recursion
    i=1
    bestcvscore=-np.inf    
    while i<=p:
        bestscore = 0
        for variable in candidates:
            score,knn = knn_test(X.iloc[:, base + [variable]], y)
            #knn.fit(X.iloc[:, base + [variable]], y)
            #score = knn.score(X.iloc[:, base + [variable]], y)
            if score > bestscore:
                bestscore = score 
                best = knn
                newvariable=variable
        base.append(newvariable)
        candidates.remove(newvariable)
        
        cvscore = cross_val_score(best, X.iloc[:, base], y, scoring='neg_mean_absolute_error').mean() 
        
        if cvscore > bestcvscore:
            bestcvscore=cvscore
            bestcv = best
            subset = base[:]
        i+=1
    bestcv_mae = -bestcvscore
    #Finalise
    return bestcv, bestcv_mae

In [ ]:
#Show correlations to have some initial KNN predictors.

pd.set_option('display.max_rows',300)
abs_correl = abs(data_num.corr().round(3)['LogSalePrice'])
abs_correl_sort = abs_correl.sort_values(ascending=False)
abs_correl_sort.head(20)

In [ ]:
abs_correl_sort.head(20).index

In [ ]:
#Create list of KNN predictors.
#Use the top correlated prodictors
#Keep some intersction terms and processsed terms rather than the original terms, because they contains more information.

predictors_knn = ['OverallQual', 'BsmtSF_Qual','GrLivArea', 'GarageArea_Cars', 'ExterQual', 'AgeHouse']

In [ ]:
#Correlation heatmap.

data_num[predictors_knn].corr().round(4)
fig, ax = plt.subplots(figsize = (10,10))
cmap = sns.diverging_palette(220,10,as_cmap=True)
sns.heatmap(data_num[predictors_knn].corr(), ax=ax, cmap=cmap)
ax.set(title='Correlation heatmap for variables used in KNN')
#fig.savefig('correlation_knn.png')

In [ ]:
#Create copy to standardize
data_k = data_num.copy()

In [ ]:
#Standardize data.

sigma=data_k[predictors_knn].std()
data_k[predictors_knn]=data_k[predictors_knn]/(2*sigma) 

In [ ]:
#Run KNN forward selection algorithm and fit model.

knn,bestcv_mae = select_knn(data_k[predictors_knn],data_k['LogSalePrice'])
print(knn)
print(bestcv_mae)

## To Make Predictions

In [ ]:
#Import validation set

data_v = pd.read_csv('test.csv')
data_v.head()

In [ ]:
#Do all the same data manipulation/processing changes as with the train data!!

#Convert to string type

data_v['MSSubClass'] = data_v['MSSubClass'].astype(str)
data_v['MoSold'] = data_v['MoSold'].astype(str)

#Fill object type blanks with NA one by one for record purposes

data_v['LotFrontage'] = data_v['LotFrontage'].fillna(0)
data_v['Alley'] = data_v['Alley'].fillna('NA')
data_v['BsmtQual'] = data_v['BsmtQual'].fillna('NA')
data_v['BsmtCond'] = data_v['BsmtCond'].fillna('NA')
data_v['BsmtExposure'] = data_v['BsmtExposure'].fillna('NA')
data_v['BsmtFinType1'] = data_v['BsmtFinType1'].fillna('NA')
data_v['BsmtFinType2'] = data_v['BsmtFinType2'].fillna('NA')
data_v['FireplaceQu'] = data_v['FireplaceQu'].fillna('NA')
data_v['GarageType'] = data_v['GarageType'].fillna('NA')
data_v['GarageFinish'] = data_v['GarageFinish'].fillna('NA')
data_v['GarageQual'] = data_v['GarageQual'].fillna('NA')
data_v['GarageCond'] = data_v['GarageCond'].fillna('NA')
data_v['PoolQC'] = data_v['PoolQC'].fillna('NA')
data_v['Fence'] = data_v['Fence'].fillna('NA')
data_v['MiscFeature'] = data_v['MiscFeature'].fillna('NA')
data_v['MasVnrArea'] = data_v['MasVnrArea'].fillna(0)
data_v['MasVnrType'] = data_v['MasVnrType'].fillna('None')
data_v['BsmtFullBath'] = data_v['BsmtFullBath'].fillna(0)

#Create age house and age garage to use in analysis

data_v['AgeHouse'] = (data_v['YrSold'] - data_v['YearRemod/Add']).astype(float)
data_v['AgeGarage'] = (data_v['YrSold'] - data_v['GarageYrBlt']).astype(float)

#Delete year built, remodeled, and garage year built and fill blank AgeGarage with 0

del data_v['YearBuilt']
del data_v['YearRemod/Add']
del data_v['GarageYrBlt']
data_v['AgeGarage'] = data_v['AgeGarage'].fillna(0)

In [ ]:
#Combine Neighborhood variables

neighbor_combined = {'MeadowV':'MeadowV',
                     'BrDale': 'BrD_IDO',
                     'IDOTRR': 'BrD_IDO',
                     'OldTown':'Old_Ed_SW_Brk',
                     'Edwards':'Old_Ed_SW_Brk',
                     'SWISU':'Old_Ed_SW_Brk',
                     'BrkSide':'Old_Ed_SW_Brk',
                     'Sawyer':'Sa_NA_Bl_NP',
                     'NAmes':'Sa_NA_Bl_NP',
                     'Blueste':'Sa_NA_Bl_NP',
                     'NPkVill':'Sa_NA_Bl_NP',
                     'Mitchel':'Mi_SaW,Bng',
                     'SawyerW':'Mi_SaW,Bng',
                     'Blmngtn':'Mi_SaW,Bng',
                     'Gilbert':'Gi_NWA_Cr_Co',
                     'NWAmes':'Gi_NWA_Cr_Co',
                     'Crawfor':'Gi_NWA_Cr_Co',
                     'CollgCr':'Gi_NWA_Cr_Co',
                     'Greens':'Gr_CC_So_Ti',
                     'ClearCr':'Gr_CC_So_Ti',
                     'Somerst':'Gr_CC_So_Ti',
                     'Timber':'Gr_CC_So_Ti',
                     'Veenker':'Veenker',
                     'StoneBr':'St_No_NHt',
                     'NoRidge':'St_No_NHt',
                     'NridgHt':'St_No_NHt'}
data_v['Neighborhood'] = data_v['Neighborhood'].map(neighbor_combined)

In [ ]:
#Convert quality ordinals to numerical

exterqual_vals = {'Fa':1,'TA':2,'Gd':3,'Ex':4}
data_v['ExterQual'] = data_v['ExterQual'].map(exterqual_vals).astype(int)

bsmt_vals = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['BsmtQual'] = data_v['BsmtQual'].map(bsmt_vals).astype(int)

bsmt_exp = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
data_v['BsmtExposure'] = data_v['BsmtExposure'].map(bsmt_exp).astype(int)

kitchen_vals = {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['KitchenQual'] = data_v['KitchenQual'].map(kitchen_vals).astype(int)

fireplace_qual = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['FireplaceQu'] = data_v['FireplaceQu'].map(fireplace_qual).astype(int)

garage_vals= {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['GarageQual'] = data_v['GarageQual'].map(garage_vals).astype(int)

heating_vals = {'Fa':1,'TA':2,'Gd':3,'Ex':4}
data_v['HeatingQC'] = data_v['HeatingQC'].map(kitchen_vals).astype(int)

garage_cond = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['GarageCond'] = data_v['GarageCond'].map(garage_cond).astype(int)

bsmt_cond = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
data_v['BsmtCond'] = data_v['BsmtCond'].map(bsmt_cond).astype(int)

In [ ]:
#Create different lists by data type.

columns_float_v = data_v.select_dtypes(['float64']).columns
columns_int_v = data_v.select_dtypes(['int64']).columns
columns_cat_v = data_v.select_dtypes(['object']).columns

In [ ]:
#Create dummy variables as before.

columns_cat_v = list(columns_cat_v)
for column in columns_cat_v:
    data_v = pd.get_dummies(data_v, columns = [column])

In [ ]:
#Create interactions as before.

data_v['BsmtSF_Qual'] = data_v['TotalBsmtSF']*data_v['BsmtQual']
data_v['GarageArea_Qual'] = data_v['GarageArea']*data_v['GarageQual']
data_v['GrLiv_Rooms'] = data_v['GrLivArea']*data_v['TotRmsAbvGrd']
data_v['GarageArea_Cars'] = data_v['GarageArea']*data_v['GarageCars']

In [ ]:
#Standardize data for KNN.

data_knn = data_v.copy()
sigma=data_num[predictors_knn].std()
data_knn[predictors_knn]=data_knn[predictors_knn]/(2*sigma) 

In [ ]:
#Standardize data for all other methods.

pred_float_v = data_v.select_dtypes(['float64']).columns
pred_int_v = data_v.select_dtypes(['int64']).columns
pred_dum_v = data_v.select_dtypes(['uint8']).columns

mu_float = np.mean(data_num[pred_float])
sigma_float = np.std(data_num[pred_float])

mu_int = np.mean(data_num[pred_int])
sigma_int = np.std(data_num[pred_int])

mu_dum = np.mean(data_num[pred_dum])
sigma_dum = np.std(data_num[pred_dum])

data_v[pred_float] = (data_v[pred_float]-mu_float)/(2*sigma_float)
data_v[pred_int] = (data_v[pred_int]-mu_int)/(2*sigma_int)
data_v[pred_dum] = (data_v[pred_dum])/(sigma_dum)

data_v.head()

In [ ]:
#Predictions based on OLS model

y_predict_ols = pd.DataFrame(np.exp(ols.predict(data_v[predictors1])))
pd.set_option('display.max_rows', 1608)
y_predict_ols

In [ ]:
#Predictions based on Lasso model

y_predict_lasso = pd.DataFrame(np.exp(lasso.predict(data_v[predictors2])))
pd.set_option('display.max_rows', 1608)
y_predict_lasso

In [ ]:
#Predictions based on Forward + Ridge model

y_predict_ridge = pd.DataFrame(np.exp(ridge_fw.predict(data_v[predictors2])))
pd.set_option('display.max_rows', 1608)
y_predict_ridge

In [ ]:
#Predictions based on ENet model

y_predict_enet = pd.DataFrame(np.exp(enet.predict(data_v[predictors2])))
pd.set_option('display.max_rows', 1615)
y_predict_enet

In [ ]:
#Predictions based on Forward + PCR model

y_predict_pcr = pd.DataFrame(np.exp(pcr.predict(data_v[predictors2])))
pd.set_option('display.max_rows', 1608)
y_predict_pcr

In [ ]:
#Predictions based on KNN model

y_predict_knn = pd.DataFrame(np.exp(knn.predict(data_knn[predictors_knn])))
pd.set_option('display.max_rows', 1608)
y_predict_knn